In [1]:
import random
import numpy as np
import pandas as pd
from pprint import pprint



## Train & Test Split

In [2]:
def train_test_split(df, train_size, random_state=0):
    # setting a random state 
    random.seed(random_state)

    # check if the testsize if fload or not
    if isinstance(train_size, float):
        if (1.0 > train_size > 0.0):
            # rounding to get the right portion for the split
            train_size = round(train_size * len(df))
        else:
            print("The Test Size that u have choosed is not applicable choose from the range of [0,1]")

    # df.index  returns RangeIndex(start=0, stop=150, step=1) // returns the indices of df
    # so we need to change that to a list to we can pick our indices
    indices = df.index.tolist()
    # picking our random sample
    train_indices = random.sample(population=indices, k=train_size)

    # df.loc assign allocated samples with specific indices to a df 
    train_df = df.loc[train_indices]
    # df.drop removes those samples from the df
    test_df = df.drop(train_indices)

    return train_df, test_df


## Leaf Purity Check

In [3]:
# to check if the data has only one classification or not
def check_purity(data):

    # access only the last column of the dataset
    label_column = data[:, -1]

    # all the uniuqe classifications are saved into one array 
    unique_classes = np.unique(label_column)

    # checking if the classification array has only one class
    if len(unique_classes) == 1:
        return True
    else:
        return False


## Data classify

In [4]:
def classify_data(data):

    label_column = data[:, -1]

    # the unique classes are saved into array and they counts into counts array
    unique_classes, counts_unique_classes = np.unique(
        label_column, return_counts=True)

    # to get the index of the class that has more samples in it
    index = counts_unique_classes.argmax()
    # returning the major class
    classification = unique_classes[index]

    return classification


## Getting the Potential Splits

In [5]:
def get_potential_splits(data):

    potenial_splits = {}  # creating an empty list
    _, n_columns = data.shape  # getting number of columns
    for column_index in range(n_columns - 1):  # loop over these columns

        # lets see those values and loop over them later
        values = data[:, column_index]
        # take only the unique values
        unique_values = np.unique(values)

        feature_types = FEATURE_TYPES[column_index]

        if feature_types == "continious":
            # creats a set with the number of columns in the data likest this : {0: [], 1: [], 2: [], 3: []}
            potenial_splits[column_index] = []
            # looping on the on the indicies of these unique values
            for index in range(len(unique_values)):
                # breaking loop condition for (unique_values[0])
                if index != 0:
                    # creating a split between those unique values
                    current_value = unique_values[index]
                    previous_value = unique_values[index - 1]
                    # if unique value are 0.2 & 0.3 the split will be at (0.3+0.2)/2 : 2.5
                    split = ((current_value + previous_value) / 2)
                    # back to the list we created : {0: [], 1: [], 2: [], 3: []}
                    # all splits will be appended here  ^      ^      ^      ^
                    # after each full loop potenial_splits should have one group filled with splits and other intiated {0: [1, 2, 3], 1: [] }
                    potenial_splits[column_index].append(split)

        else:
            potenial_splits[column_index] = unique_values

    return potenial_splits


## Data splitter using class

In [6]:
def split_data(data, split_column, split_value):

    # split_column is a colunm in our data [ feature values]
    # split_value is a value choosen to split that column

    split_column_values = data[:, split_column]

    feature_types = FEATURE_TYPES[split_column]

    if feature_types == "continious":
        data_below = data[split_column_values <= split_value]
        data_above = data[split_column_values > split_value]

    else:
        data_below = data[split_column_values == split_value]
        data_above = data[split_column_values != split_value]

    return data_below, data_above


## Entropy

In [7]:
def caclulated_entropy(data):

    label_column = data[:, -1]
    _, counts = np.unique(label_column, return_counts=True)

    probabilities = counts / counts.sum()   # p(A) = #of A / total samples 
    entorpy = sum(probabilities * -np.log2(probabilities)) # entropy low

    return entorpy


In [8]:
def caclulated_Overall_entropy(data_below, data_above):

    n_data_points = len(data_below) + len(data_above)

    p_data_below = len(data_below) / n_data_points
    p_data_above = len(data_above) / n_data_points

    overall_entropy = (p_data_below * caclulated_entropy(data_below)
                       + p_data_above * caclulated_entropy(data_above))

    return overall_entropy


## Choosing the Best Spliter

In [9]:
def determine_best_split(data, potential_splits):

    # a way to determine the best split is by measuring the intropy of the that split
    overall_entropy = 999
    # loop over the potential splits {set}
    for column_index in potential_splits:
        # loop over the values of the splits in each column inside the set
        for value in potential_splits[column_index]:
            # split data and measure the overall entropy
            data_below, data_above = split_data(
                data, split_column=column_index, split_value=value)
            current_overall_entropy = caclulated_Overall_entropy(
                data_below, data_above)
            # choose the split with the lowest overall entropy
            if current_overall_entropy <= overall_entropy:
                overall_entropy = current_overall_entropy
                best_split_column = column_index
                best_split_value = value

    return best_split_column, best_split_value


## Determining the Feature Type 

In [10]:
def Determine_feature_type(df):
    
    feature_types = []
    n_unique_values_threshold = 15

    for column in df.columns:
        unique_values = df[column].unique()
        example_value = unique_values[0]

        if(isinstance(example_value, str) or len(unique_values)<= n_unique_values_threshold):
            feature_types.append("categorical")

        else:
            feature_types.append("continious")

    return feature_types

## Decsion Tree Algorithm

In [11]:
def decision_tree_algorthim(df, counter=0, min_samples=2, max_depth=5):


    
    # data_preparation
    if counter == 0:
        # save the feature names and feature type [ categorical or continious]

        # since at default we want work with np arrays so values are changed
        data = df.values
        global COLUMN_HEADERS
        COLUMN_HEADERS = df.columns
        global FEATURE_TYPES
        FEATURE_TYPES = Determine_feature_type(df)

    else:
        # as the counter goes up for the recursion part data is saves as df to enter the recursion
        data = df

    # returning the classification ither when there only
        # 1- data is pure
        # 2 - pruning
    if (check_purity(data)) or (len(data) < min_samples) or (counter == max_depth):
        classification = classify_data(data)
        return classification

    # recursive part
    else:
        # when data is not pure or no pruning is happening counter is increased so data can be splited again
        counter += 1

        # re splitting of the data to classify it
        potential_splits = get_potential_splits(data)
        split_column, split_value = determine_best_split(
            data, potential_splits)
        data_below, data_above = split_data(data, split_column, split_value)

        # creation of tree
        # choosing the best feature for the decsion - again the split_columns are the features
        feature_name = COLUMN_HEADERS[split_column]
        feature_types = FEATURE_TYPES[split_column]

        if feature_types == "continious":
            # creating the question that will indicate the classification
            question = "{} <= {} ".format(feature_name, split_value)

        else:
            question = "{} == {} ".format(feature_name, split_value)
        # the sub_tree is a set , branch , so over all either imagine it as
        # 1- tree with branches
        # 2- set with sub_sets
        sub_tree = {question: []}

        # creating the left and right path for the tree - and recurse on them
        # true on the left ,,,, False on the right
        yes_answer = decision_tree_algorthim(
            data_below, counter, min_samples, max_depth)
        no_answer = decision_tree_algorthim(
            data_above, counter, min_samples, max_depth)

        # incase of the rght and left answers are both the same thing we replace the sub_tree with just a leaf
        if yes_answer == no_answer:
            sub_tree = yes_answer
        else:
            # appedning the subtree to the main tree
            sub_tree[question].append(no_answer)
            sub_tree[question].append(yes_answer)


        return sub_tree


## Sample classifier

In [12]:
def classify_sample(example, tree):

    question = list(tree.keys())[0]
    feature_name, comarsio_opperator, value = question.split()

    # check if the feature contained in that classification or not

    if comarsio_opperator == "<=":
        if example[feature_name] <= float(value):
            # if yes go left [answer = yes_answer]
            answer = tree[question][0]
        else:
            # if no go right [answer = no_answer]
            answer= tree[question][1]

    else: 
        if str(example[feature_name]) == value:
            # if yes go left [answer = yes_answer]
            answer = tree[question][0]
        else:
            # if no go right [answer = no_answer]
            answer= tree[question][1]

    # base case
    # check if the answer is a dictionary value or not 
    # if it's a dict value classification is returned 
    if not isinstance(answer, dict):
        return answer

    # if not recurse till you get only a dictionary value
    # recursion part
    else:
    # cut tree and recurse in the sub_tree
        residual_tree = answer
        return classify_sample(example, residual_tree)

## Accuracy

In [13]:
def get_accuracy (df, tree):

    # accessing the last column in the dataframe and save it
    last_column = df[df.columns[-1]]

    # creating a new data frame for classification with the classes only
    classification_df = pd.DataFrame(last_column)

    # adding a column of predictions to the dataframe
    classification_df["classification"] = df.apply(
        classify_sample, axis=1, args=(tree,), )

    # check the predictions
    classification_df["classification_correct"] = classification_df.iloc[:,0] == classification_df.iloc[:, 1]
    # calculating the accuracy
    accuracy = classification_df.classification_correct.mean()
    
        
    return accuracy


## Tree printer

In [14]:
def print_tree(tree):

    pprint(tree)
